# ViT for mineral phases quantification from XRD patterns  
A 4 classes problem, i.e 4 mineral phases: Calcite, Dolomite, Gibbsite and Hematite  
Quantify each mineral phase from an XRD pattern. 

In [1]:
from functools import partial
from netrc import netrc
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU
from torch.nn import CrossEntropyLoss
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
import pickle
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
#from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import torch.optim as optim
import copy
import torch.nn.functional as F
import datetime
import os
import time

## GPU

In [2]:
if torch.cuda.is_available():
    print("CUDA is available.")
    print("PyTorch version:", torch.__version__)
    print("CUDA version:", torch.version.cuda)
    print("Number of available GPUs:", torch.cuda.device_count())
    print("GPU name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

CUDA is not available.


## Dataloader

In [3]:
"""
classes = ['Calcite','Gibbsite','Dolomite','Hematite']
path = 'Databases/Proportion'

class Diffractogramm(Dataset):
    def __init__(self, train = 'train'):
        self.train = train
        if (self.train == 'train'):
            self.data_prob = torch.tensor(pd.read_csv(path+'label_train.txt',sep = ',').values).to(device)
            self.data_file = torch.tensor(pd.read_csv(path+'Data_train.txt', sep = ',', header = None).values).to(device)
        if (self.train == 'val'):
            self.data_prob = torch.tensor(pd.read_csv(path+'label_val.txt',sep=',').values).to(device)
            self.data_file = torch.tensor(pd.read_csv(path+'Data_val.txt', sep = ',', header = None).values).to(device)

        
    def __len__(self):
        return self.data_prob.shape[0]

    def __getitem__(self, idx):
        I = self.data_file[idx,:]
        prop = self.data_prob[idx,:]
        I = I.float()
        sample = [I,prop]
        return sample
        
#Create the datasets/dataloader (train & validation)
batch_size_train = 128
batch_size_val = 128
trainset = Diffractogramm(train = 'train')
valset = Diffractogramm(train = 'val')
dataloader = DataLoader(trainset, batch_size = batch_size_train, shuffle = True, num_workers=0)
validloader = DataLoader(valset, batch_size = batch_size_val , shuffle = True, num_workers=0)
"""

"\nclasses = ['Calcite','Gibbsite','Dolomite','Hematite']\npath = 'Databases/Proportion'\n\nclass Diffractogramm(Dataset):\n    def __init__(self, train = 'train'):\n        self.train = train\n        if (self.train == 'train'):\n            self.data_prob = torch.tensor(pd.read_csv(path+'label_train.txt',sep = ',').values).to(device)\n            self.data_file = torch.tensor(pd.read_csv(path+'Data_train.txt', sep = ',', header = None).values).to(device)\n        if (self.train == 'val'):\n            self.data_prob = torch.tensor(pd.read_csv(path+'label_val.txt',sep=',').values).to(device)\n            self.data_file = torch.tensor(pd.read_csv(path+'Data_val.txt', sep = ',', header = None).values).to(device)\n\n        \n    def __len__(self):\n        return self.data_prob.shape[0]\n\n    def __getitem__(self, idx):\n        I = self.data_file[idx,:]\n        prop = self.data_prob[idx,:]\n        I = I.float()\n        sample = [I,prop]\n        return sample\n        \n#Create the

## ViT block

In [4]:
""" *** FUNCTIONS*** """
class EncoderBlock(nn.Module):
    def __init__(self,
                 dim,
                 num_heads,
                 mlp_ratio=4.,
                 qkv_bias=False,
                 qk_scale=None,
                 drop_ratio=0.,
                 attn_drop_ratio=0.,
                 drop_path_ratio=0.,
                 act_layer=nn.GELU,
                 norm_layer=nn.LayerNorm):
        super(EncoderBlock, self).__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
                              attn_drop_ratio=attn_drop_ratio, proj_drop_ratio=drop_ratio)
        self.drop_path = DropPath(
            drop_path_ratio) if drop_path_ratio > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim,
                       act_layer=act_layer, drop=drop_ratio)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

class Mlp(nn.Module):

    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class Attention(nn.Module):

    def __init__(self,
                 dim,
                 num_heads=2,
                 qkv_bias=False,
                 qk_scale=None,
                 attn_drop_ratio=0.,
                 proj_drop_ratio=0.):
        super(Attention, self).__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop_ratio)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop_ratio)

    def forward(self, x):
        # [batch_size, num_patches + 1, total_embed_dim]
        B, N, C = x.shape
        # qkv(): -> [batch_size, num_patches + 1, 3 * total_embed_dim]
        # reshape: -> [batch_size, num_patches + 1, 3, num_heads, embed_dim_per_head]
        # permute: -> [3, batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        # [batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        # make torchscript happy (cannot use tensor as tuple)
        q, k, v = qkv[0], qkv[1], qkv[2]
        # transpose: -> [batch_size, num_heads, embed_dim_per_head, num_patches + 1]
        # @: multiply -> [batch_size, num_heads, num_patches + 1, num_patches + 1]
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        # @: multiply -> [batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        # transpose: -> [batch_size, num_patches + 1, num_heads, embed_dim_per_head]
        # reshape: -> [batch_size, num_patches + 1, total_embed_dim]
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x
        
        

def Spectra_Embedding(x, spec_length, embed_dim):
    batch_size = x.shape[0]
    new_spec_length = (spec_length // embed_dim) * embed_dim
    x = x[:, :new_spec_length]
    x = torch.reshape(x, (batch_size, spec_length // embed_dim, embed_dim))
    return x

        
class VIT(nn.Module):
    def __init__(self, spec_length=2000, num_output=1,
                 embed_dim=40, depth=12, num_heads=2, mlp_ratio=4.0, qkv_bias=True,
                 qk_scale=None, drop_ratio=0.,
                 attn_drop_ratio=0., drop_path_ratio=0., norm_layer=None,
                 act_layer=None):
        
        # MSTransformer
        super(VIT, self).__init__()
        self.num_classes = num_output
        self.spec_length = spec_length
        self.num_features = self.embed_dim = embed_dim
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        act_layer = act_layer or nn.GELU
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, (spec_length//embed_dim) + 1, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_ratio)
        dpr = [x.item() for x in torch.linspace(0, drop_path_ratio, depth)]
        self.blocks = nn.Sequential(*[
            EncoderBlock(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                        drop_ratio=drop_ratio, attn_drop_ratio=attn_drop_ratio, drop_path_ratio=dpr[i],
                        norm_layer=norm_layer, act_layer=act_layer)
            for i in range(depth)
        ])
        self.norm = norm_layer(embed_dim)
        
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        self.apply(_init_vit_weights)

        self.head = nn.Linear(embed_dim, num_output)

    def forward(self, x):
        # [B , xrd_length] --> [B , xrd_length/embed_dim , embed_dim]
        x = Spectra_Embedding(x, self.spec_length, self.embed_dim)
        cls_token = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        x = self.pos_drop(x + self.pos_embed)
        x = self.blocks(x)
        x = self.norm(x)
        x = self.head(x)

        return x[:, 0]
    
def _init_vit_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.trunc_normal_(m.weight, std=.01)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LayerNorm):
        nn.init.zeros_(m.bias)
        nn.init.ones_(m.weight)

def VIT_model(spec_length=2905,num_output: int = 1):
    model = VIT(spec_length=spec_length,
                              embed_dim=80,
                              depth=12,
                              num_heads=2,
                              num_output=num_output)
    return model

## Loss function: MSE & Dirichlet

In [5]:
"""
Definition of a class (nn.Module) for the 5 loss function :
    - forward method is used during the training (loss function)
    - prediction method is used for validation and test set like a predictor

*** PARAMETERS *** (for each function)
    - a is the output of the neural network
    - p is the proportion vector from the ground truth
"""

#Loss 1 : DIR & MSE
class LossDirichlet_MSE(nn.Module):
    def forward(self, a , p):
        b,K = a.shape # batch size and number of classes
        alpha = F.relu(a) + 1.0 #phi function of the paper
        S = torch.sum(alpha, dim = 1, keepdim = True) # sum of alpha
        moment_1 = alpha / S #moment of order 1 for Dirichlet Distribution
        moment_2 = alpha*(1+alpha) / (S*(1+S)) #moment of order 2 for Dirichlet Distribution
        L = torch.sum((p*p - 2*p*moment_1 + moment_2), dim = 1)
        L = torch.mean(L)
        return(L)
    
    def prediction(self,a):
        output = F.relu(a) + 1.0
        S = torch.sum(output,dim = 1, keepdim = True)
        output = output/S
        return(output)

## Train function 

Use of Tensorboard  
Return the trained model, epoch with the best MMAE

In [6]:
def train(model, nb_epoch, criterion, eps, N_train, num_classes):
    writer = SummaryWriter(log_dir = 'Tensorboard')
    norm_val = np.inf
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(),lr = 0.0001)
    n_train = len(trainset)
    iter_info = 100
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(nb_epoch):
        since_epoch = time.time()
        running_loss = 0.0
        epoch_loss = 0.0
        model.train()
        for i, data in enumerate(dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            images = data[0]
            proportion = data[1]
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(images)

            loss = criterion(outputs.float(),proportion.float())
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_loss += loss.item() * batch_size_train
            if i % iter_info == (iter_info-1):    # print every iter_info mini-batches
                print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / iter_info))
                running_loss = 0.0
        time_elapsed_epoch = time.time() - since_epoch
        print('Epoch completed in {:.0f}m {:.0f}s'.format(
            time_elapsed_epoch // 60, time_elapsed_epoch % 60))
        epoch_loss = epoch_loss / n_train
        print('Epoch loss: %.3f' % (epoch_loss))
        writer.add_scalar('Epoch_loss/train', epoch_loss, epoch)

        #Validation set
        norme_inf = 0
        norme_2 = 0
        good_support = 0
        with torch.no_grad():
            model.eval()
            for data in validloader:
                images = data[0]
                p = data[1]
                output = model(images)
                output = criterion.prediction(output)
                
                predictive_support = output > eps
                true_support = (p > 0)
                for m in range(predictive_support.shape[0]):
                    if torch.equal(true_support[m],predictive_support[m]):
                        good_support = good_support + 1
                
                Diff_abs = torch.abs(p-output)
                norme_inf = norme_inf + torch.sum(torch.amax(Diff_abs,axis = 1))
                norm = (p-output)**2
                norme_2 = norme_2 + torch.sum(torch.mean(norm,axis = 1))

            norme_inf = norme_inf / len(valset)
            norme_2 = norme_2 / len(valset)
            RMSE = torch.sqrt(norme_2)
            supp = good_support/len(valset)
            
            writer.add_scalar('MMAE',norme_inf,epoch)
            writer.add_scalar('RRS', supp,epoch)
            writer.add_scalar('RMSE',RMSE,epoch)
            
            print(str(criterion))
            print('Epoch: ',epoch+1,', MMAE : ', norme_inf)
            print('RMSE : ',RMSE)
            print('RRS : ',supp)
            
            if (norme_inf < norm_val):
                norm_val = norme_inf
                best_model_wts = copy.deepcopy(model.state_dict())
                print('Model epoch is better')
                    
    print('Finished Training')
    time_elapsed = time.time() - since
    print('Training completed in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
    
    model.load_state_dict(best_model_wts)
    return(model)




## Train the model

In [7]:
"""
nb_epoch = 1000
eps = 1e-2
num_classes=4
nb_train = 5
for w in range(nb_train):    
    model = VIT_model(spec_length=2905,num_output=num_classes)
    model = model.to(device)
    criterion = LossDirichlet_MSE()
    print('Train '+ str(w) +' :Dirichlet & MSE')
    model_train = train(model = model, nb_epoch = nb_epoch, criterion = criterion, eps = eps, N_train = w, num_classes = num_classes)
    torch.save(model_train.state_dict(),f = 'Trained_model')
"""    

"\nnb_epoch = 1000\neps = 1e-2\nnum_classes=4\nnb_train = 5\nfor w in range(nb_train):    \n    model = VIT_model(spec_length=2905,num_output=num_classes)\n    model = model.to(device)\n    criterion = LossDirichlet_MSE()\n    print('Train '+ str(w) +' :Dirichlet & MSE')\n    model_train = train(model = model, nb_epoch = nb_epoch, criterion = criterion, eps = eps, N_train = w, num_classes = num_classes)\n    torch.save(model_train.state_dict(),f = 'Trained_model')\n"

## Test the trained model on 32 real data

### Load the real data

In [8]:
"""
***Classes definition and path to data***
"""
classes = ['Calcite','Gibbsite','Dolomite','Hematite']
path = 'Databases/Real_data/'

real_datalist = pd.read_csv(path+'Labels_T.csv', sep=';',header=None,names=['file','Calcite','Gibbsite',
                                        'Dolomite','Hematite'],skiprows=1)

Data = torch.zeros(32,2905)
for i in range(32):
    V = pd.read_csv(path+real_datalist['file'][i],header = None, skiprows = 0,
                    names = ['angle','intensity'],sep = ',')
    V_tensor = np.asarray(V['intensity'])
    V_tensor = torch.tensor(V_tensor).float()
    if (len(V_tensor) == 2905):
        Data[i,:] = V_tensor
        
        
 
"""
Data pre-processing
"""
Data_m_2 = np.array(Data)
for i in range(len(Data_m_2)):
    m = min(Data_m_2[i,:])
    Data_m_2[i,:] = Data_m_2[i,:] - m
    Data_m_2[i,:] = Data_m_2[i,:]/max(Data_m_2[i,:])
Data_M_2 = torch.tensor(Data_m_2)
print(Data_M_2.shape)

torch.Size([32, 2905])


### Test function

In [9]:
"""
Function to test the trained neural network on the pixels of the Data
*** PARAMETERS ***
- model = trained neural network
- output_file = file for write the output (RMSE, Infinite norm, %Support)
- criterion : criterion use for trained NN (use "out" method)
"""

def test (model, criterion):
    model_ft = model
    norme_inf = 0
    norme_2 = 0
    norme_2_classes = np.zeros(len(classes))
    good_support = 0
    eps = 7e-2
    with torch.no_grad():
        model_ft.eval()
        for i in range(len(Data_M_2)):
            #Load the data
            images = Data_M_2[i,:]
            images = torch.unsqueeze(images,0)
            
            #Load the label
            prop_Calcite = real_datalist.loc[i,'Calcite']
            prop_Gibbsite = real_datalist.loc[i,'Gibbsite']
            prop_Dolomite = real_datalist.loc[i,'Dolomite']
            prop_Hematite = real_datalist.loc[i,'Hematite']
            p = torch.tensor([prop_Calcite,prop_Gibbsite,prop_Dolomite,prop_Hematite])
            
            #Results after NN
            y = model_ft(images)
            res = criterion.prediction(y)
            res = res.squeeze(0)
            #print(res)
            
            #Support
            predictive_support = res > eps
            true_support = (p > 0)
            if np.array_equal(true_support,predictive_support):
                good_support = good_support + 1
            
            #MMAE
            Diff_abs = torch.abs(p - res)
            Diff_abs = np.array(Diff_abs)
            norme_inf = norme_inf + np.amax(Diff_abs)
            
            #RMSE
            norm = np.array((p-res)**2)
            norme_2 = norme_2 + np.mean(norm)
            norme_2_classes = norme_2_classes + norm
            
        MMAE = norme_inf / len(Data)
        norme_2 = norme_2 / len(Data)
        RMSE = np.sqrt(norme_2)
        RMSE_classes = np.sqrt( norme_2_classes / len(Data))
        
        supp = good_support / len(Data)

        
    
        return (MMAE, RMSE, supp, RMSE_classes)

In [10]:
num_classes = len(classes)
model_ft = VIT_model(spec_length=2905,num_output=num_classes)
criterion = LossDirichlet_MSE()
model_ft.load_state_dict(torch.load('Trained_model/Prop' ,map_location=torch.device('cpu')))
MMAE, RMSE, Supp, RMSE_classes = test(model = model_ft, criterion = criterion)


In [11]:
print('MMAE: ', MMAE)
print('RMSE: ', RMSE)
print('RRS: ', Supp)

MMAE:  0.05810671030944077
RMSE:  0.04561921506860975
RRS:  0.96875
